## Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../scripts')))

In [5]:
df = pd.read_json('../data/result.json')

In [6]:
df.head()

,name,type,id,messages
0,Vent Here,public_channel,1137849815,"{'id': 1, 'type': 'service', 'date': '2017-05-..."
1,Vent Here,public_channel,1137849815,"{'id': 2, 'type': 'message', 'date': '2017-05-..."
2,Vent Here,public_channel,1137849815,"{'id': 3, 'type': 'message', 'date': '2017-05-..."
3,Vent Here,public_channel,1137849815,"{'id': 4, 'type': 'message', 'date': '2017-05-..."
4,Vent Here,public_channel,1137849815,"{'id': 5, 'type': 'message', 'date': '2017-05-..."


In [7]:
df_message = df[['messages']]

In [8]:
df_message.head()

,messages
0,"{'id': 1, 'type': 'service', 'date': '2017-05-..."
1,"{'id': 2, 'type': 'message', 'date': '2017-05-..."
2,"{'id': 3, 'type': 'message', 'date': '2017-05-..."
3,"{'id': 4, 'type': 'message', 'date': '2017-05-..."
4,"{'id': 5, 'type': 'message', 'date': '2017-05-..."


In [9]:
df_message.iloc[1]['messages']

{'id': 2,
 'type': 'message',
 'date': '2017-05-27T17:36:54',
 'date_unixtime': '1495895814',
 'edited': '2023-01-19T02:15:20',
 'edited_unixtime': '1674083720',
 'from': 'Vent Here',
 'from_id': 'channel1137849815',
 'text': ["A place to ask for advice, look for comfort or simply ramble on about what's on your mind with no judgement.\n\nSend ur thoughts to your admin: ",
  {'type': 'mention', 'text': '@unihorse'}],
 'text_entities': [{'type': 'plain',
   'text': "A place to ask for advice, look for comfort or simply ramble on about what's on your mind with no judgement.\n\nSend ur thoughts to your admin: "},
  {'type': 'mention', 'text': '@unihorse'}]}

In [10]:
# Extract the keys and values from the 'messages' column
messages_list = df_message['messages'].apply(pd.Series)

# Create a new dataframe from the extracted keys and values
df_messages_expanded = pd.DataFrame(messages_list)

# Display the new dataframe
df_messages_expanded.head()

,id,type,date,date_unixtime,actor,actor_id,action,title,text,text_entities,...,sticker_emoji,mime_type,width,height,duration_seconds,message_id,photo,forwarded_from,performer,poll
0,1,service,2017-05-27T17:17:47,1495894667,Vent Here,channel1137849815,create_channel,Vent Here,,[],...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,message,2017-05-27T17:36:54,1495895814,NaN,NaN,NaN,NaN,"[A place to ask for advice, look for comfort o...","[{'type': 'plain', 'text': 'A place to ask for...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,message,2017-05-27T18:37:51,1495899471,NaN,NaN,NaN,NaN,Hey unihorse 🐴.\nHide my identity.\nI have a c...,"[{'type': 'plain', 'text': 'Hey unihorse 🐴. Hi...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,message,2017-05-27T18:48:49,1495900129,NaN,NaN,NaN,NaN,Hey unihorse 🐴\nHide my identity.\nI got a con...,"[{'type': 'plain', 'text': 'Hey unihorse 🐴 Hid...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,message,2017-05-28T05:44:35,1495939475,NaN,NaN,NaN,NaN,Hey unihorse 🐴.\nPlease hide my identity.\nHer...,"[{'type': 'plain', 'text': 'Hey unihorse 🐴. Pl...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_filtered = df_messages_expanded.set_index('id')[['date', 'text']]
df_filtered.head()

,date,text
id,,
1,2017-05-27T17:17:47,
2,2017-05-27T17:36:54,"[A place to ask for advice, look for comfort o..."
3,2017-05-27T18:37:51,Hey unihorse 🐴.\nHide my identity.\nI have a c...
4,2017-05-27T18:48:49,Hey unihorse 🐴\nHide my identity.\nI got a con...
5,2017-05-28T05:44:35,Hey unihorse 🐴.\nPlease hide my identity.\nHer...


In [12]:
df_filtered = df_filtered.drop([1, 2])
df_filtered.head()

,date,text
id,,
3,2017-05-27T18:37:51,Hey unihorse 🐴.\nHide my identity.\nI have a c...
4,2017-05-27T18:48:49,Hey unihorse 🐴\nHide my identity.\nI got a con...
5,2017-05-28T05:44:35,Hey unihorse 🐴.\nPlease hide my identity.\nHer...
6,2017-05-28T08:36:58,Hey unihorse 🐴.\nHide my identity.\nJust venti...
8,2017-05-28T09:55:31,Hey unihorse 🐴.\nHide my identity.\nIm so rich...


## Removing Emojis from the text column

In [13]:
import re

def remove_emojis(text):
    if not isinstance(text, str):
        return text
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # emoticons
        "\U0001F300-\U0001F5FF"  # symbols & pictographs
        "\U0001F680-\U0001F6FF"  # transport & map symbols
        "\U0001F1E0-\U0001F1FF"  # flags (iOS)
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

df_filtered['text'] = df_filtered['text'].apply(remove_emojis)
df_filtered.head()

,date,text
id,,
3,2017-05-27T18:37:51,Hey unihorse .\nHide my identity.\nI have a co...
4,2017-05-27T18:48:49,Hey unihorse \nHide my identity.\nI got a conf...
5,2017-05-28T05:44:35,Hey unihorse .\nPlease hide my identity.\nHere...
6,2017-05-28T08:36:58,Hey unihorse .\nHide my identity.\nJust ventin...
8,2017-05-28T09:55:31,Hey unihorse .\nHide my identity.\nIm so rich....


In [ ]:
df_filtered['text'] = df_filtered['text'].str.replace(r'Hey unihorse \.\nHide my identity\.\n', '', regex=True)
df_filtered.head()

,date,text
id,,
3,2017-05-27T18:37:51,I have a confession to make.\nI am a very free...
4,2017-05-27T18:48:49,Hey unihorse \nHide my identity.\nI got a conf...
5,2017-05-28T05:44:35,Hey unihorse .\nPlease hide my identity.\nHere...
6,2017-05-28T08:36:58,Just venting:\nTrying to set your friend up wi...
8,2017-05-28T09:55:31,"Im so rich. Im not bragging, im just so rich, ..."


In [ ]:
df_filtered['text'] = df_filtered['text'].str.replace(r'Hey unihorse \nHide my identity\.\n', '', regex=True)
df_filtered.head()

,date,text
id,,
3,2017-05-27T18:37:51,I have a confession to make.\nI am a very free...
4,2017-05-27T18:48:49,I got a confession to make am a dude with diff...
5,2017-05-28T05:44:35,Hey unihorse .\nPlease hide my identity.\nHere...
6,2017-05-28T08:36:58,Just venting:\nTrying to set your friend up wi...
8,2017-05-28T09:55:31,"Im so rich. Im not bragging, im just so rich, ..."


In [14]:
from langdetect import detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException

In [17]:
DetectorFactory.seed = 0
def is_english(text):
    if not isinstance(text, str):
        return False
    try:
        return detect(text) == "en"
    except LangDetectException:  # Handle empty or invalid text
        return False

# Apply the function to filter messages
df_filtered['is_english'] = df_filtered['text'].apply(is_english)
df_english = df_filtered[df_filtered['is_english']].drop(columns=['is_english'])

# Display filtered DataFrame
print("Filtered DataFrame with only English messages:")
print(df_english)


Filtered DataFrame with only English messages:
                      date                                               text
id                                                                           
3      2017-05-27T18:37:51  Hey unihorse .\nHide my identity.\nI have a co...
4      2017-05-27T18:48:49  Hey unihorse \nHide my identity.\nI got a conf...
5      2017-05-28T05:44:35  Hey unihorse .\nPlease hide my identity.\nHere...
6      2017-05-28T08:36:58  Hey unihorse .\nHide my identity.\nJust ventin...
8      2017-05-28T09:55:31  Hey unihorse .\nHide my identity.\nIm so rich....
...                    ...                                                ...
16276  2022-10-25T17:12:47  Our bot is currently inactive, routine mainten...
16450  2022-11-03T19:19:31  Concerning the "Request identity" feature on o...
17088  2022-12-08T20:06:02  Twenty six years ago, a women was rushed into ...
17089  2022-12-08T20:30:46  And this happens to be his all time favourite ...
17130  2022-12-20

In [18]:
df_english.to_csv('../data/filtered_messages.csv', index=False)